In [319]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import Ridge,SGDRegressor,Lasso,LinearRegression
from sklearn.metrics import r2_score,mean_absolute_error,make_scorer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE

In [258]:
data=pd.read_csv("C:\\Users\\SUDARKODI S\\Downloads\\NPPE1_ModelBuilding3.csv")

In [260]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.087472,0.002829,1.768235,0.188396,0.117071,0.148148,0.478165,0.720443,0.967195,0.281563,0.428571,0.959596,1.0,0.0,23.358097
1,0.378379,0.022079,1.115629,0.091974,0.066089,0.851852,0.911759,0.785321,0.885001,0.424648,0.285714,0.868687,0.0,1.0,17.268768
2,0.066901,0.003828,-0.536262,0.221188,0.255671,0.296296,0.228024,0.406472,0.980184,0.274376,0.428571,0.767677,0.0,1.0,27.776974
3,0.140645,0.011132,1.323366,0.422514,0.153103,0.148148,0.410679,0.200319,0.861371,0.305006,0.142857,0.848485,0.0,1.0,16.121960
4,0.144225,0.204918,-0.930790,0.148694,0.177490,0.259259,0.146832,0.111429,0.983448,0.286322,0.285714,0.616162,1.0,0.0,23.129426


2. Split the dataset into train dataset and test dataset in the following manner
	Use train_test_split to split the dataset into train and test dataset with test size equal to 0.3(i.e.30%) and random_state equal to 42. Let other parameters have default values.
	Columns except the last column should be the feature matrix (X_train or X_test)
	Last column will be the label vector.

(Common instructions for Q.2, Q.3 and Q.4)

Train the ridge model on the training data with the following parameters:

alpha = 10
solver = 'saga'
tol = 1e-4
random_state = 42

Enter the value of 
𝑅
2
R 
2
   score on the test dataset.

In [265]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.3)

In [195]:
ridge=Ridge(alpha=10,solver='saga',tol=1e-4,random_state=42)
ridge.fit(X_train,y_train)

Ridge(alpha=10, random_state=42, solver='saga')

In [196]:
y_pred=ridge.predict(X_test)
r2=r2_score(y_test,y_pred)
r2

0.6613547575262211

What is the index of most important feature? Note the index starts from 0. Ignore the intercept for this question.

In [198]:
# larger the coefficient the most important 
# smaller the coefficient , least importance

In [199]:
ridge.coef_

array([ -0.4825441 ,   3.74601838,  -0.73583331,   0.54199933,
        -9.89014109,   5.80114296,  -5.06099736,  -9.45015598,
         4.73124885, -23.51321982,  11.31863371,   0.49450664,
        -0.89196134,   0.89196134])


What is the index of least important feature? Note the index starts from 0. Ignore the intercept for this question.

In [201]:
small=-23.51321982

(Common Instructions for Q.5 and Q.6)
Take SGDRegressor(random_state = 42) estimator with GridSearchCV. Hyperparameter tuning to be done over the following parameters:1. 
penalty as ['l1', 'l2'2. ]
alpha values as [1e-5, 1e-4, 1e-3, 1e-2, 1e-3. 1]
values of tol as [1e-4, 1e-3, 1e-2, 1e4. -1]
Use cross-validation5.  = 5
Set scoring as neg_mean_absolute_error
Use the best model from above hyper parameter tuning process to answer following questions:

What is the best penalty?

In [204]:
sgd=SGDRegressor(random_state=42)

param_grid={
    'penalty' : ['l1', 'l2'],
    'alpha' : [1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    'tol' :  [1e-4, 1e-3, 1e-2, 1e-1]
}

scorer = make_scorer(mean_absolute_error, greater_is_better=False)

In [205]:
grid=GridSearchCV(estimator=sgd,param_grid=param_grid,cv=5,scoring=scorer)

In [206]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=SGDRegressor(random_state=42),
             param_grid={'alpha': [1e-05, 0.0001, 0.001, 0.01, 0.1],
                         'penalty': ['l1', 'l2'],
                         'tol': [0.0001, 0.001, 0.01, 0.1]},
             scoring=make_scorer(mean_absolute_error, greater_is_better=False, response_method='predict'))

In [207]:
grid.best_estimator_

SGDRegressor(alpha=0.001, random_state=42, tol=0.0001)

In [208]:
grid.best_params_

{'alpha': 0.001, 'penalty': 'l2', 'tol': 0.0001}

What will be value of mean absolute error on the test dataset?

In [210]:
grid_y_predict=grid.predict(X_test)

In [211]:
mean_absolute_error(y_test,grid_y_predict)

3.8131121797994014

(Common Instructions for Q.7 and Q.8)
Create a pipeline of the PCA() as transformer and Lasso as an estimator.
Use GridSearchCV for tuning the hyperparameters of the created pipeline on training dataset1. 
	Values of n_components for PCA to be [0.9, 0.92. ]
	lasso alpha value to be taken as : [10, 1, 0.01, 0.03. 1]
	scoring : neg_mean_absolute_er4. or.
	Use 5 fold cross valida5. ion.
	n_jobs = -1 (negative one) [it helps in using all the computational power to run this job]
(Note: Kindly ignore the warning.)

If we fit the pipeline on the training dataset, what will be the R2 score on the test dataset?

In [267]:
pipe=Pipeline([
    ('pca',PCA()),
    ('lasso',Lasso())
] )

In [269]:
param_grid = {
'pca__n_components': [0.9, 0.95],
'lasso__alpha': [10, 1, 0.01, 0.001]
}

scorer = make_scorer(r2_score)

In [216]:
LassoGrid=GridSearchCV(estimator=pipe,param_grid=param_grid,cv=5,scoring=scorer,n_jobs=-1)

In [217]:
LassoGrid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pca', PCA()), ('lasso', Lasso())]),
             n_jobs=-1,
             param_grid={'lasso__alpha': [10, 1, 0.01, 0.001],
                         'pca__n_components': [0.9, 0.95]},
             scoring=make_scorer(mean_absolute_error, greater_is_better=False, response_method='predict'))

In [218]:
LassoGrid.best_params_

{'lasso__alpha': 0.01, 'pca__n_components': 0.95}

In [219]:
grid_predict=LassoGrid.predict(X_test)
r2=r2_score(y_test,grid_predict)

In [220]:
r2

0.6288625430197571

How much variance is explained (Eigen value) by the first principle component?


In [271]:
grid=GridSearchCV(estimator=pipe,param_grid=param_grid,cv=5,scoring='neg_mean_absolute_error',n_jobs=-1)

In [273]:
grid.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pca', PCA()), ('lasso', Lasso())]),
             n_jobs=-1,
             param_grid={'lasso__alpha': [10, 1, 0.01, 0.001],
                         'pca__n_components': [0.9, 0.95]},
             scoring='neg_mean_absolute_error')

In [275]:
grid.best_estimator_.named_steps

{'pca': PCA(n_components=0.95), 'lasso': Lasso(alpha=0.01)}

In [277]:
pca=grid.best_estimator_.named_steps['pca']
pca.fit(X_train)

PCA(n_components=0.95)

In [284]:
pca.explained_variance_[0]

1.1635075742239032

Create a pipeline of the PolynomialFeatures as transformer and Lasso as an estimator with the following parameters:
  - For PolynomialFeatures:
    - interaction_only = False
    - degree = 2
  - For Lasso:
    - alpha = 1
    - warm_start = True
    - random state as 0

Fit the pipeline on the training dataset and find the 
𝑅
2
R 
2
  score on the test dataset.

In [296]:
pipe=Pipeline([
    ('poly',PolynomialFeatures(interaction_only=False,degree=2)),
    ('lasso',Lasso(alpha=1,warm_start=True,random_state=0))
])

In [298]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('poly', PolynomialFeatures()),
                ('lasso', Lasso(alpha=1, random_state=0, warm_start=True))])

In [300]:
poly_predict=pipe.predict(X_test)

In [302]:
r2_score(y_test,poly_predict)

0.157678032410551

If you eliminate 1 feature with recursive feature elimination, which feature will be eliminated?
Type the index of the eliminated feature (index starts from 0).
Use 
1
LinearRegression
 model with default parameters as an estimator.
Use processed training data.

In [309]:
estimator=LinearRegression()

In [313]:
rfe=RFE(estimator=estimator,n_features_to_select=X_train.shape[1]-1)

In [315]:
rfe.fit(X_train,y_train)

RFE(estimator=LinearRegression(), n_features_to_select=13)

In [321]:
np.where(rfe.support_ == False)[0][0]

2